In [ ]:
# default_exp workflow
%load_ext lab_black
# nb_black if running in jupyter
%load_ext autoreload
# automatically reload python modules if there are changes in the
%autoreload 2

In [ ]:
# hide
from nbdev.showdoc import *

# Workflow

> Define workflow for automatically updating, training and deploying your ML model!


***input:*** optional

***output:*** python script for execiting the workflow

***description:***

Define your automated workflow. You can either define static or dynamic workflow, or create a main loop routine and API for using your ML model.

Edit this and other text cells to describe your project. 

Remember that you can utilize the `#export` tag to export cell commands to `[your_module]/workflow.py`.

## Import relevant modules

In [ ]:
# export
import papermill
from pathlib import Path
import os

# your code here

## Define notebook parameters

In [ ]:
# your code here
seed = 0

make direct derivations from the paramerters:

In [ ]:
# your code here

## Define workflow

Here we present a tiny example you can try running yourself and then extend to your needs.
For more complex workflows, consider utilizing the Snakemake tool!

In [ ]:
# export

"""
A workflow to re-run your machine learning workflow automatically.

The script will:
- rebuild your python module
- run data notebook to reload and clean data
- run model notebook to sw test your model
- run loss notebook to train and evaluate your model with full data,
    and save or deploy it for further use

Feel free to edit!
"""

cwd = Path().cwd()
save_notebooks_to = cwd / "results" / "notebooks"
# Hint! you can also create time or setup -stamped folders to store your results!

# make sure changes are updated to module
os.system("nbdev_build_lib")

# run workflow
for notebook in ["00_data.ipynb", "01_model.ipynb", "02_loss.ipynb"]:
    papermill.execute_notebook(
        notebook,  # this notebook will be executed
        save_notebooks_to
        / ("_" + notebook),  # this is where the executed notebook will be saved
        # (notebooks named with '_' -prefix are ignored by nbdev)
        parameters={"seed": 1},  # you can change notebook parameters
        kernel_name="python39myenv",
    )  # note: change kernel according to your project setup!

Executing:   0%|          | 0/29 [00:00<?, ?cell/s]

Executing:   0%|          | 0/25 [00:00<?, ?cell/s]

Executing:   0%|          | 0/32 [00:00<?, ?cell/s]

You can also define your workflow in another language than Python and write it into a file from this notebook.
This way your script is still included the documentation without copy-pasting.
You can also add the script to .gitignore to avoid double tracking.

In [ ]:
%%writefile workflow.sh
# do not edit the workflow.sh file directly!
# make all edits to 03_workflow.ipynb and re-run the notebook to update script!

# your code here

Overwriting workflow.sh


You can also run bash and R scripts from inside a notebook:

In [ ]:
%%bash
./workflow.sh

## You can now move on to API notebook to define an interface to use your trained model!